In [0]:
%sql
create catalog if not exists dev2
managed location 'abfss://unity-catalog-storage@dbstorageffygriwrcxej6.dfs.core.windows.net/1140122096371529' 

In [0]:
%sql
create database if not exists coures_ws.demodb

In [0]:
%sql
drop database if exists dev2.default cascade
    

In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""
flight_time_dt = spark.read.format('json').schema(flight_schema_ddl).option("dateFormat", "M/d/y").load('abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time.json')


In [0]:
display(flight_time_dt)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS coures_ws.demodb.flight_time_tbl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
) USING DELTA

LOAD DATA INTO DELTA TABLE

In [0]:
flight_time_dt.write.format("delta").mode("append").saveAsTable("coures_ws.demodb.flight_time_tbl")

In [0]:
flight_time_tbl = spark.read.table("coures_ws.demodb.flight_time_tbl")
display(flight_time_tbl)

CREATE A DELTA TABLE USING DELTATABLE API

In [0]:
(spark.read
    .format("delta")
    .load("abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time")
    .display()
)



from delta import DeltaTable

(DeltaTable.createOrReplace(spark)
    .tableName("coures_ws.demodb.flight_time_tbl")
    .addColumn("id", "INT")
    .addColumn("FL_DATE", "DATE")
    .addColumn("OP_CARRIER", "STRING")
    .addColumn("OP_CARRIER_FL_NUM", "INT")
    .addColumn("ORIGIN", "STRING")
    .addColumn("ORIGIN_CITY_NAME", "STRING")
    .addColumn("DEST", "STRING") 
    .addColumn("DEST_CITY_NAME", "STRING")
    .addColumn("CRS_DEP_TIME", "INT")
    .addColumn("DEP_TIME", "INT")
    .addColumn("WHEELS_ON", "INT")
    .addColumn("TAXI_IN", "INT")
    .addColumn("CRS_ARR_TIME", "INT")
    .addColumn("ARR_TIME", "INT")
    .addColumn("CANCELLED", "STRING")
    .addColumn("DISTANCE", "INT")
    .execute()
)

In [0]:
%python
# Read the file into a DataFrame
df = spark.read.format("delta").option("header", "true").load("abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time")

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("flight_time")

# Query the temporary view using SQL
result = spark.sql("SELECT * FROM flight_time")

# Display the result
display(result)

In [0]:
(flight_time_dt.coalesce(1)
    .write
    .format("delta")
    .mode("overwrite")
    .save("abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time")
)

In [0]:
%python
dbutils.fs.ls("abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time/part-00000-84d9fbd7-b79c-41ea-99ad-5820fba3f4b7-c000.snappy.parquet")

READ DELTA TABLE USING SQL AND DATAFRAME API

In [0]:
%sql
select * from coures_ws.demodb.flight_time_tbl

In [0]:
spark.read.format("delta").table("coures_ws.demodb.flight_time_tbl").display()

READ EXTERNAL DATA USING DATAFRAM API

In [0]:
(spark.read
    .format("delta")
    .load("abfss://dbfs-container@peterbassey.dfs.core.windows.net/external/CH7-Data Files/dataset_ch7/flight-time")
    .display()
)

DELETE ONE RECORD FROM TABLE USING DELTATABLE API

In [0]:
from delta import DeltaTable

people_dt = DeltaTable.forName(spark, "coures_ws.demodb.people") if DeltaTable.isDeltaTable(spark, "coures_ws.demodb.people") else None
if people_dt is not None: people_dt.delete("firstName = 'abdul'")